In [69]:
def convert_to_df(target_url):
    df = pd.read_csv(target_url, sep='\s+')
    df = df[1:]
    df = df.drop(['WVHT', 'DPD', 'APD', 'MWD', 'PRES', 'WTMP', 'PTDY', 'TIDE', 'GST'], axis=1)
    df = df.replace(['MM'],'nan')
    rest = df.iloc[:,5:].astype(float)
    first = df.iloc[:,:5]
    df = pd.concat([first, rest],axis=1)
    df.rename(columns = {'#YY':'YY'},inplace=True)
    df['date'] = df['YY']+'-'+df['MM']+'-'+df['DD']
    df['time'] = df['hh']+':'+df['mm']
    df['datetime'] = pd.to_datetime(df['date']+' '+df['time'])
    df=df[1:]
    df.reset_index(drop = True, inplace = True)
    return df

In [9]:
from datetime import datetime, timedelta
def datetime_range(start, end, delta):
    current = start
    while current < end:
        yield current
        current += delta

In [73]:
def generate_datetime(df):
    start_time = df['datetime'][len(df)-1]
    end_time = df['datetime'][0]
    dts = [dt.strftime('%Y-%m-%d %H:%M:%S') for dt in 
       datetime_range(start_time.to_pydatetime(), end_time.to_pydatetime(),timedelta(minutes=20))]
    dts.sort(reverse = True)
    df2=pd.DataFrame()
    df2['datetime'] = pd.to_datetime(dts)
    df1 = df2.merge(df, on='datetime', how='left')
    return df1

In [76]:
def final_df(target_url):
    df = convert_to_df(target_url)
    df1 = generate_datetime(df)
    df1.interpolate(limit_direction="both",inplace=True)
    df1 = df1.drop(['YY', 'MM', 'DD', 'hh', 'mm', 'date', 'time'], axis=1)
    df1.set_index('datetime', inplace = True)
    df1 = df1[::-1]
    return df1

In [77]:
df = final_df(target_url)

In [78]:
df

,WDIR,WSPD,ATMP,DEWP,VIS
datetime,,,,,
2021-12-02 22:35:00,230.0,1.5,21.0,16.0,8.7
2021-12-02 22:55:00,230.0,0.0,21.0,16.0,8.7
2021-12-02 23:15:00,230.0,0.0,21.0,16.0,8.7
2021-12-02 23:35:00,230.0,0.0,21.0,16.0,8.7
2021-12-02 23:55:00,230.0,0.0,21.0,16.0,8.7
...,...,...,...,...,...
2021-12-21 18:15:00,300.0,3.6,16.0,11.0,8.7
2021-12-21 18:35:00,300.0,2.6,16.0,10.0,8.7
2021-12-21 18:55:00,300.0,3.1,16.0,10.0,8.7


In [17]:
url2 = "https://www.ndbc.noaa.gov/data/realtime2/KBQX.txt"

In [79]:
df2 = final_df(url2)

In [80]:
df2

,WDIR,WSPD,ATMP,DEWP,VIS
datetime,,,,,
2021-12-15 00:15:00,130.0,5.7,23.0,22.0,5.2
2021-12-15 00:35:00,130.0,5.7,23.0,22.0,5.2
2021-12-15 00:55:00,130.0,5.7,23.0,22.0,4.3
2021-12-15 01:15:00,130.0,5.7,23.0,22.0,5.2
2021-12-15 01:35:00,130.0,5.7,23.0,22.0,5.2
...,...,...,...,...,...
2022-01-29 00:15:00,10.0,10.3,14.0,-1.0,8.7
2022-01-29 00:35:00,350.0,10.3,14.0,-1.0,8.7
2022-01-29 00:55:00,360.0,11.3,14.0,-1.0,8.7


In [81]:
url3 = "https://www.ndbc.noaa.gov/data/realtime2/KMIS.txt"

In [82]:
df3 = final_df(url3)

In [83]:
df3

,WDIR,WSPD,ATMP,DEWP,VIS
datetime,,,,,
2021-12-15 00:15:00,120.0,5.1,22.0,19.0,8.7
2021-12-15 00:35:00,120.0,6.2,22.0,19.0,8.7
2021-12-15 00:55:00,100.0,7.2,22.0,19.0,8.7
2021-12-15 01:15:00,110.0,8.2,22.0,19.0,8.7
2021-12-15 01:35:00,120.0,7.7,22.0,19.0,8.7
...,...,...,...,...,...
2022-01-29 15:15:00,20.0,11.3,6.0,-4.0,8.7
2022-01-29 15:35:00,30.0,13.9,6.0,-4.0,8.7
2022-01-29 15:55:00,20.0,10.8,6.0,-6.0,8.7
